# Model-Free Analysis

This notebook will investigate the fixation distributions generated by examples/example4_new/addm_gen_data.ipynb using model independent analyses as described by Eum et al. (2023).

### Load the data

In [24]:
import pickle
import numpy as np

data = pickle.load(open("examples/example4_new/new_addm_data_20260202-022735.pkl", "rb"))

DATA_TYPE = np.float64

a = data["a"]
b = data["b"]
x0 = data["x0"]
eta_true = data["eta"]
kappa_true = data["kappa"]
r1_data = data["r1_data"]
r2_data = data["r2_data"]
flag_data = data["flag_data"].astype(np.int32)

sigma = data["sigma"]
T = data["T"]

mu1_true_data = kappa_true * (r1_data - eta_true * r2_data)
mu2_true_data = kappa_true * (eta_true * r1_data - r2_data)

mu_true_data = data["mu_array_padded_data"].astype(DATA_TYPE)
sacc_data = data["sacc_array_padded_data"].astype(DATA_TYPE)
length_data = data["d_data"].astype(np.int32)
rt_data = data["decision_data"][:, 0].astype(DATA_TYPE)
choice_data = data["decision_data"][:, 1].astype(np.int32)

num_data, max_d = mu_true_data.shape

In Eum et al. (2023), drift is possibly established as 3 ($\frac{0.003}{0.001}$), theta as 0.5, and noise as about 0.7 ($\frac{0.022}{\sqrt{0.001}}$). In efficient-fpt, these variables are formalized as drift $\mapsto$ kappa, theta $\mapsto$ eta, and noise is still sigma. The transformation is direct, so this makes analysis simple. What is difficult is the way saccades are formalized within the model. Liu et al. (2025) ignore fixations on the basis that they are instantaneous. Thus, the array representing saccades is just a flag for instantaneous change for fixations. Stages are left-inclusive right-exclusive.

A useful thing to have would be a mapping from the `sacc_data`, `mu_true_data` to fixation tuples given a dt. `sacc_data` is measured in seconds and padded with 0's after the trial ends up to a length of 13. Similarly, `mu_true_data` conveniently provides relevant readings for drift rate for a given stage padded with 0's after the trial ends up to a length of 13. 

Now, given the large eta in this simulated trial, it is possible for evidence to accumulate in a direction opposite to attention. Thus, we use flag to find the location to which the first fixation is directed. There is no simulated left fixation first bias within flag. In a sample of 100000 trials, 50030 were left fixation first.

In [25]:
from math import ceil, floor

def expand_fixations(sacc_data, flag_data, rt_data, dt):
    """
    Parameters
    ----------
    sacc_data : list of 1D np.ndarray
        saccade times per trial (seconds)
    flag_data : 1D np.ndarray
        initial fixation per trial (0/1 or similar)
    rt_data : 1D np.ndarray
        reaction time per trial (seconds)
    dt : float
        timestep size (seconds)

    Returns
    -------
    array
        Each element is a tuple of fixation locations for one trial
    """
    all_trials = []

    for saccs, start_fix, rt in zip(sacc_data, flag_data, rt_data):

        # number of discrete timesteps (inclusive rt)
        fix_len = int(floor(rt / dt)) + 1

        # initialize fixation array
        fix = np.full(fix_len, start_fix)

        if len(saccs) > 0:
            # convert saccade times to indices
            switch_idxs = [int(ceil(s / dt)) for s in saccs]

            # apply alternating flips
            for idx in switch_idxs[1:]:
                if idx >= fix_len or idx <= 0:
                    continue
                fix[idx:] = 1 - fix[idx]

        all_trials.append(tuple(int(x) for x in fix))

    return all_trials

In [26]:
expanded_fixations = expand_fixations(sacc_data, flag_data, rt_data, 0.001)

In [27]:
print(f'Representative fixations: {expanded_fixations[0][::400]}') 
print(f"Corresponding drift rates: {[f'{mu:.2f}' for mu in mu_true_data[0] if mu != 0]}")
print(f'Length of fixations: {len(expanded_fixations[0])}')
print(f'True RT: {rt_data[0]}')

Representative fixations: (0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0)
Corresponding drift rates: ['0.95', '-0.10', '0.95', '-0.10', '0.95', '-0.10', '0.95']
Length of fixations: 4174
True RT: 4.17319500001118


The inverse will also be useful to migrate old data into new.

In [28]:
def compress_fixations(fixation_trials, max_d, dt):
    """
    Inverse of expand_fixations (up to dt resolution),
    with zero-padded saccade arrays, using
    left-inclusive / right-exclusive convention.

    Parameters
    ----------
    fixation_trials : iterable of tuple[int]
        Output of expand_fixations
    dt : float
        timestep size (seconds)
    max_d : int
        maximum number of saccades per trial (including padding)

    Returns
    -------
    sacc_data : list of np.ndarray, shape (max_d,)
        Zero-padded saccade times (seconds)
    flag_data : np.ndarray
        Initial fixation per trial
    rt_data : np.ndarray
        Reaction times per trial (seconds)
    """
    sacc_data = []
    flag_data = []
    rt_data = []

    eps = 1e-12

    for fix in fixation_trials:
        fix = np.asarray(fix, dtype=int)

        # initial fixation
        flag_data.append(fix[0])

        # reaction time (inclusive)
        rt_data.append((len(fix) - 1) * dt)

        # detect fixation switches (indices)
        switch_idxs = np.where(fix[1:] != fix[:-1])[0] + 1

        # map indices -> times inside ((k-1)dt, kdt]
        sacc_times = switch_idxs * dt - eps

        # prepend true start time (left-inclusive)
        sacc_times = np.insert(sacc_times, 0, 0.0)

        # pad with zeros up to max_d
        padded = np.zeros(max_d, dtype=float)
        n = min(len(sacc_times), max_d)
        padded[:n] = sacc_times[:n]

        sacc_data.append(padded)

    return sacc_data, np.array(flag_data), np.array(rt_data)


In [29]:
tsacc_data, tflag_data, trt_data = compress_fixations(expanded_fixations, max_d, 0.001)

In [35]:
print(f'Transformed sacc_data: {tsacc_data[0]}')
print(f"True sacc_data: {[f'{sacc:.4f}' for sacc in sacc_data[0]]}")

Transformed sacc_data: [0.    1.022 1.799 2.187 2.903 3.335 3.729 0.    0.    0.    0.    0.
 0.   ]
True sacc_data: ['0.0000', '1.0214', '1.7983', '2.1863', '2.9021', '3.3346', '3.7287', '0.0000', '0.0000', '0.0000', '0.0000', '0.0000', '0.0000']


As we see, the transformation incorporate left-inclusive and right-exclusive timing. However, the inverse transformation is only able to recover timing up to dt. Since the model makes no claim about sub dt-timings (researchers may choose to change dt depending on their needs), we can move forward assuming that both functions transform efficient-fpt data into PyDDM data and vice versa.